In [1]:
import pandas as pd

# Link Iris-Commune-Canton
File for canton: https://www.data.gouv.fr/s/resources/circonscriptions-legislatives-table-de-correspondance-des-communes-et-des-cantons-pour-les-elections-legislatives-de-2012-et-sa-mise-a-jour-pour-les-elections-legislatives-2017/20170411-141128/Table_de_correspondance_circo_legislatives2017-1.xlsx  
File for iris: https://www.insee.fr/fr/statistiques/fichier/3137409/base-ic-evol-struct-pop-2014.zip

In [6]:
df_iris = pd.read_excel('./data/base-ic-evol-struct-pop-2014.xls', sheet_name='IRIS', header=5)

In [8]:
df_cantons = pd.read_excel('./data/Table_de_correspondance_circo_legislatives2017-1.xlsx', header=0)

In [17]:
df_cantons = df_cantons[~df_cantons['CODE DPT'].astype(str).str.contains('Z')].reset_index(drop=True)

In [18]:
#to print departements and see which one to remove
#df_iris['DEP'].unique()

array([u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09',
       u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18',
       u'19', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28',
       u'29', u'2A', u'2B', u'30', u'31', u'32', u'33', u'34', u'35',
       u'36', u'37', u'38', u'39', u'40', u'41', u'42', u'43', u'44',
       u'45', u'46', u'47', u'48', u'49', u'50', u'51', u'52', u'53',
       u'54', u'55', u'56', u'57', u'58', u'59', u'60', u'61', u'62',
       u'63', u'64', u'65', u'66', u'67', u'68', u'69', u'70', u'71',
       u'72', u'73', u'74', u'75', u'76', u'77', u'78', u'79', u'80',
       u'81', u'82', u'83', u'84', u'85', u'86', u'87', u'88', u'89',
       u'90', u'91', u'92', u'93', u'94', u'95', u'971', u'972', u'973',
       u'974'], dtype=object)

In [21]:
df_iris = df_iris[~df_iris['DEP'].isin([u'2A',u'2B' u'971', u'972', u'973',
       u'974'])]

In [61]:
df_cantons['CODE DPT'] = df_cantons['CODE DPT'].astype('str').str.zfill(2)
df_cantons['CODE COMMUNE'] = df_cantons['CODE COMMUNE'].astype('str').str.zfill(3)
df_cantons['COM'] = df_cantons['CODE DPT'] + df_cantons['CODE COMMUNE']
df_cantons['CODE CANTON'] = df_cantons['CODE CANTON'].astype('str').str.zfill(2)
df_cantons['ID_CANTON'] = df_cantons['CODE DPT'] + df_cantons['CODE CANTON']

In [62]:
df_cross = pd.merge(df_iris, df_cantons, on=['COM'], how='left')

# Analyse if there is difference between each reference (iris file and canton file)
We get 6% difference between the two reference commune name (there is problem with Paris)

In [64]:
viz = df_cross[['ID_CANTON', 'COM', 'IRIS', 'NOM CANTON', 'NOM COMMUNE']].sort_values(['ID_CANTON', 'COM', 'IRIS'])

In [118]:
viz.head()

,ID_CANTON,COM,IRIS,NOM CANTON,NOM COMMUNE
1,0101,01002,010020000,Ambérieu-en-Bugey,L'Abergement-de-Varey
2,0101,01004,010040101,Ambérieu-en-Bugey,Ambérieu-en-Bugey
3,0101,01004,010040102,Ambérieu-en-Bugey,Ambérieu-en-Bugey
4,0101,01004,010040201,Ambérieu-en-Bugey,Ambérieu-en-Bugey
5,0101,01004,010040202,Ambérieu-en-Bugey,Ambérieu-en-Bugey


In [114]:
df_cross['MEME NOM COMMUNE'] = (df_cross['LIBCOM'].isin(df_cross['NOM COMMUNE']) & df_cross['NOM COMMUNE'].isin(df_cross['LIBCOM']))

In [107]:
df_cross['MEME NOM COMMUNE'].value_counts()

True     74652
False     4580
Name: MEME NOM COMMUNE, dtype: int64

In [117]:
df_cross[['ID_CANTON', 'COM', 'IRIS', 'NOM CANTON', 'NOM COMMUNE']][df_cross['MEME NOM COMMUNE'] == False]

,ID_CANTON,COM,IRIS,NOM CANTON,NOM COMMUNE
131,0118,01095,010950000,Saint-Etienne-du-Bois,Nivigne et Suran
185,0114,01152,011520000,Nantua,Echallon
186,0120,01153,011530000,Thoiry,Echenevex
187,0102,01154,011540000,Attignat,Etrez
188,0110,01155,011550000,Hauteville-Lompnes,Evosges
202,0114,01170,011700000,Nantua,Beard-Géovreissiat
204,NaN,01172,011720000,NaN,NaN
349,NaN,01316,013160000,NaN,NaN
368,0104,01338,013380000,Belley,Groslée-Saint-Benoît
378,0112,01349,013490000,Meximieux,Saint-Eloi


# Complete population information

When we compare these informations https://fr.wikipedia.org/wiki/Liste_des_cantons_de_la_Sarthe
with our dataframe sarthe_canton we  got the exact same things

In [127]:
canton_pop = df_cross.groupby('ID_CANTON')['P14_POP'].sum().reset_index()

In [128]:
sarthe_canton = canton_pop[canton_pop['ID_CANTON'].str[:2] == '72']
sarthe_canton

,ID_CANTON,P14_POP
1504,7201,29709.0
1505,7202,28767.0
1506,7203,21717.0
1507,7204,27778.0
1508,7205,24979.0
1509,7206,24408.0
1510,7207,29813.0
1511,7208,28963.0
1512,7209,26561.0
1513,7210,289979.0


In [130]:
canton_to_cut = canton_pop[canton_pop['P14_POP'] > 60000]

In [132]:
len(canton_to_cut)

440